In [8]:
import utils
import ast
import time
from copy import deepcopy
import random
import math

In [9]:
input_file = f'inputs/dummy.yaml'
output_file = f'outputs/my_dummy.txt'

# Incarca fisierul YAML
input_text = utils.read_yaml_file(input_file)

# Convertim sirurile de caractere de la intervale in tupluri de int-uri
for i, interval in enumerate(input_text['Intervale']):
    input_text['Intervale'][i] = ast.literal_eval(interval)

print(input_text)

{'Intervale': [(8, 10), (10, 12), (12, 14)], 'Materii': {'DS': 100, 'IA': 75, 'MS': 100}, 'Profesori': {'Andreea Dinu': {'Constrangeri': ['Luni', 'Marti', '!Miercuri', '!8-10', '!12-14', '10-12'], 'Materii': ['DS', 'IA']}, 'Cristina Dumitrescu': {'Constrangeri': ['!Luni', 'Marti', '!Miercuri', '!10-12', '8-10', '12-14'], 'Materii': ['MS', 'DS']}, 'Elena Gheorghe': {'Constrangeri': ['Luni', '!Marti', 'Miercuri', '!10-14', '8-10'], 'Materii': ['DS']}, 'Pavel Filipescu': {'Constrangeri': ['Luni', '!Marti', 'Miercuri', '!8-10', '10-14'], 'Materii': ['IA']}, 'Roxana Gheorghe': {'Constrangeri': ['Luni', 'Marti', 'Miercuri', '!10-12', '8-10', '12-14'], 'Materii': ['MS', 'DS']}}, 'Sali': {'EG324': {'Capacitate': 25, 'Materii': ['MS', 'IA']}, 'EG390': {'Capacitate': 25, 'Materii': ['DS']}}, 'Zile': ['Luni', 'Marti', 'Miercuri']}


In [10]:
# Impartim dictionarul initial in mai multe dictionare

intervale = input_text['Intervale']
materii = list(input_text['Materii'].keys())
profesori = list(input_text['Profesori'].keys())
sali = list(input_text['Sali'].keys())
zile = input_text['Zile']

# Dictionarul pentru a stoca relatia materie - profesori
materii_profesori = {}

# Iteram prin fiecare profesor
for profesor, info_profesor in input_text['Profesori'].items():
    # Iteram prin fiecare materie pe care o preda profesorul respectiv
    for materie in info_profesor['Materii']:
        # Daca materia exista in dictionar, adaugam profesorul la lista de profesori asociata acelei materii
        if materie in materii_profesori:
            materii_profesori[materie].append(profesor)
        # Altfel, cream o noua intrare in dictionar cu materie si profesorul
        else:
            materii_profesori[materie] = [profesor]

print(f"Intervale: {intervale}")
print(f"Materii: {materii}")
print(f"Profesori: {profesori}")
print(f"Sali: {sali}")
print(f"Zile: {zile}")
print(f"Materii - Profesori: {materii_profesori}")

Intervale: [(8, 10), (10, 12), (12, 14)]
Materii: ['DS', 'IA', 'MS']
Profesori: ['Andreea Dinu', 'Cristina Dumitrescu', 'Elena Gheorghe', 'Pavel Filipescu', 'Roxana Gheorghe']
Sali: ['EG324', 'EG390']
Zile: ['Luni', 'Marti', 'Miercuri']
Materii - Profesori: {'DS': ['Andreea Dinu', 'Cristina Dumitrescu', 'Elena Gheorghe', 'Roxana Gheorghe'], 'IA': ['Andreea Dinu', 'Pavel Filipescu'], 'MS': ['Cristina Dumitrescu', 'Roxana Gheorghe']}


In [11]:
# FUNCTIE AUXILIARA

def parse_interval(interval : str):
    '''
    Se parsează un interval de forma "Ora1 - Ora2" în cele 2 componente.
    '''

    intervals = interval.split('-')
    return int(intervals[0].strip()), int(intervals[1].strip())

In [12]:
class State:
    def __init__(
        self,
        board: dict[str, dict[tuple[int, int], dict[str, tuple[str, str]]]] | None = None,
        conflicts: int | None = None,
    ) -> None:

        self.board = board if board is not None else State.generate_board()
        self.nconflicts = conflicts if conflicts is not None \
            else State.compute_conflicts(self.board, input_text)

    @staticmethod
    def generate_board() -> dict[str, dict[tuple[int, int], dict[str, tuple[str, str]]]]:
        '''
        Construieste un orar gol
        '''
        # Construim starea initiala
        initial_state = {}

        # Iteram prin fiecare zi
        for ziua in zile:
            initial_state[ziua] = {}
            
            # Iteram prin fiecare interval
            for interval in intervale:
                initial_state[ziua][interval] = {}
                
                # Iteram prin fiecare sala
                for sala in sali:
                        initial_state[ziua][interval][sala] = None

        return initial_state

    @staticmethod
    def compute_conflicts(timetable: dict[str, dict[tuple[int, int], dict[str, tuple[str, str]]]], timetable_specs : dict) -> int:
        '''
        Calculeaza numarul de conflicte parcurgand orarul
        '''
        constrangeri_incalcate = 0

        acoperire_target = timetable_specs['Materii']
        
        acoperire_reala = {subject : 0 for subject in acoperire_target}

        ore_profesori = {prof : 0 for prof in timetable_specs['Profesori']}

        # CONSTRANGERI HARD

        for day in timetable:
            for interval in timetable[day]:
                profs_in_crt_interval = []
                for room in timetable[day][interval]:
                    if timetable[day][interval][room]:
                        prof, subject = timetable[day][interval][room]
                        acoperire_reala[subject] += timetable_specs['Sali'][room]['Capacitate']

                        # PROFESORUL PREDA 2 MATERII IN ACELASI INTERVAL
                        if prof in profs_in_crt_interval:
                            print(f'Profesorul {prof} preda 2 materii in acelasi interval!')
                            constrangeri_incalcate += 10000
                        else:
                            profs_in_crt_interval.append(prof)

                        ore_profesori[prof] += 1

        # CONDITIA DE ACOPERIRE
        for subject in acoperire_target:
            if acoperire_reala[subject] < acoperire_target[subject]:
                print(f'Materia {subject} nu are acoperirea necesară!')
                constrangeri_incalcate += 10000 - acoperire_reala[subject]

        # CONDITIA DE MAXIM 7 ORE PE SAPTAMANA
        for prof in ore_profesori:
            if ore_profesori[prof] > 7:
                print(f'Profesorul {prof} tine mai mult de 7 sloturi!')
                constrangeri_incalcate += 10000

        #######################################################################################
                
        # CONSTRANGERI SOFT
                
        for prof in timetable_specs['Profesori']:
            for const in timetable_specs['Profesori'][prof]['Constrangeri']:
                if const[0] != '!':
                    continue
                else:
                    const = const[1:]

                    if const in timetable_specs['Zile']:
                        day = const
                        if day in timetable:
                            for interval in timetable[day]:
                                for room in timetable[day][interval]:
                                    if timetable[day][interval][room]:
                                        crt_prof, _ = timetable[day][interval][room]
                                        if prof == crt_prof:
                                            print(f'Profesorul {prof} nu dorește să predea în ziua {day}!')
                                            constrangeri_incalcate += 1
                    
                    elif '-' in const:
                        interval = parse_interval(const)
                        start, end = interval

                        if start != end - 2:
                            intervals = [(i, i + 2) for i in range(start, end, 2)]
                        else:
                            intervals = [(start, end)]


                        for day in timetable:
                            for interval in intervals:
                                if interval in timetable[day]:
                                    for room in timetable[day][interval]:
                                        if timetable[day][interval][room]:
                                            crt_prof, _ = timetable[day][interval][room]
                                            if prof == crt_prof:
                                                print(f'Profesorul {prof} nu dorește să predea în intervalul {interval}!')
                                                constrangeri_incalcate += 1

        return constrangeri_incalcate

    def conflicts(self) -> int:
        '''
        Intoarce numarul de conflicte din aceasta stare.
        '''
        return self.nconflicts

    def get_next_states(self) -> list['State']:
        '''
        Intoarce un generator cu toate posibilele stari urmatoare
        '''
        cur_timetable = self.board
        stari_urmatoare = []

        # STARILE URMATOARE, INLOCUIND INTRARILE CU VALORI IN 'None' SI PE CELE CU VALORI 'None' IN VALORI

        for zi, intervale in cur_timetable.items():
            for interval, sali in intervale.items():
                for sala, tuplu in sali.items():
                    if tuplu is not None:
                        stare_urmatoare = self.clone()
                        stare_urmatoare.board[zi][interval][sala] = None
                        stare_urmatoare.nconflicts = stare_urmatoare.compute_conflicts(stare_urmatoare.board, input_text)
                        stari_urmatoare.append(stare_urmatoare)
                    else:
                        materii_sala = input_text['Sali'][sala]['Materii']
                        for materie in materii_sala:
                            for profesor in materii_profesori[materie]:
                                stare_urmatoare = self.clone()
                                stare_urmatoare.board[zi][interval][sala] = (profesor, materie)
                                stare_urmatoare.nconflicts = stare_urmatoare.compute_conflicts(stare_urmatoare.board, input_text)
                                stari_urmatoare.append(stare_urmatoare)

        return stari_urmatoare

    def clone(self) -> 'State':
        '''
        Cloneaza orarul
        '''
        return State(deepcopy(self.board), self.nconflicts)

timetable : {str : {(int, int) : {str : (str, str)}}}

timetable : {ziua : {(interval1, interval2) : {sala : (profesor, materie)}}}

In [13]:
# HILL CLIMBING
start = time.time()

max_iters = 1000

iters, states = 0, 0
new_board = State.generate_board()
state = State(board=new_board)

while iters < max_iters:
    iters += 1

    stare_minima = state
    conflicts_minim = state.conflicts()

    stari_vecine = state.get_next_states()

    for stare_vecina in stari_vecine:
        states += 1

        conflicts_stare_vecina = stare_vecina.nconflicts

        if conflicts_stare_vecina < conflicts_minim:
            stare_minima = stare_vecina
            conflicts_minim = conflicts_stare_vecina

    if conflicts_minim == state.conflicts():
        break

    state = stare_minima

end = time.time()
exec_time = end - start

board_string = utils.pretty_print_timetable(state.board, input_file)

with open(output_file, 'w') as file:
    file.write(f"{board_string}\n")
    file.write(f"Timp: {exec_time}\n")
    file.write(f"Cost conflicte: {state.nconflicts}\n")
    file.write(f"Stari explorate: {states}")

Materia DS nu are acoperirea necesară!
Materia IA nu are acoperirea necesară!
Materia MS nu are acoperirea necesară!
Materia DS nu are acoperirea necesară!
Materia IA nu are acoperirea necesară!
Materia MS nu are acoperirea necesară!
Profesorul Cristina Dumitrescu nu dorește să predea în ziua Luni!
Materia DS nu are acoperirea necesară!
Materia IA nu are acoperirea necesară!
Materia MS nu are acoperirea necesară!
Materia DS nu are acoperirea necesară!
Materia IA nu are acoperirea necesară!
Materia MS nu are acoperirea necesară!
Profesorul Andreea Dinu nu dorește să predea în intervalul (8, 10)!
Materia DS nu are acoperirea necesară!
Materia IA nu are acoperirea necesară!
Materia MS nu are acoperirea necesară!
Profesorul Pavel Filipescu nu dorește să predea în intervalul (8, 10)!
Materia DS nu are acoperirea necesară!
Materia IA nu are acoperirea necesară!
Materia MS nu are acoperirea necesară!
Profesorul Andreea Dinu nu dorește să predea în intervalul (8, 10)!
Materia DS nu are acoperi

In [14]:
state.compute_conflicts(state.board, input_text)

0